In [1]:
!pip install conllu


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
with open('train.conllu', 'r') as f:
  text=f.read()

In [2]:
from conllu import parse, parse_tree
t=parse(text)

In [3]:
print(t[1][0]['upostag'])

ADV


In [4]:
data=[]

In [5]:
len(t)

2902

In [6]:
for i in t:
  data_list=[]
  for r in i:
    data_list.append((r['form'],r['upostag']))
  data.append(data_list)

In [7]:
import nltk

In [8]:
unigram_tagger = nltk.tag.UnigramTagger(data)

In [9]:
from pythainlp.tag import PerceptronTagger

In [10]:
pt_tagger = nltk.tag.PerceptronTagger(load=False)
pt_tagger.train(data)

In [11]:
pt_tagger.tag(['ผม','รัก','คุณ','นะ','ครับ'])

[('ผม', 'PRON'),
 ('รัก', 'VERB'),
 ('คุณ', 'PRON'),
 ('นะ', 'PART'),
 ('ครับ', 'PART')]

In [12]:
with open('test.conllu', 'r') as f:
  test_text=f.read()
test_t=parse(test_text)
test_data=[]
for i in test_t:
  data_list=[]
  for r in i:
    data_list.append((r['form'],r['upostag']))
  test_data.append(data_list)

In [13]:
tagged_test_sentences = pt_tagger.tag_sents([[token for token,tag in sent] for sent in test_data])
gold = [str(tag) for sentence in test_data for token,tag in sentence]
pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]
from sklearn import metrics
print(metrics.classification_report(gold, pred))

              precision    recall  f1-score   support

         ADJ       0.60      0.53      0.57       197
         ADP       0.89      0.88      0.88       560
         ADV       0.69      0.73      0.71       521
         AUX       0.78      0.84      0.81       421
       CCONJ       0.85      0.81      0.83       270
         DET       0.82      0.94      0.87       144
        NOUN       0.92      0.94      0.93      2310
         NUM       0.97      0.94      0.95       118
        PART       0.86      0.78      0.82       169
        PRON       0.86      0.93      0.90       350
       PROPN       0.86      0.75      0.80       295
       PUNCT       1.00      1.00      1.00       125
       SCONJ       0.84      0.73      0.78       335
         SYM       1.00      1.00      1.00         1
        VERB       0.88      0.87      0.88      1867

    accuracy                           0.86      7683
   macro avg       0.85      0.84      0.85      7683
weighted avg       0.86   

In [14]:
import json
def convert_model(name,data):
  newdata = {}
  newdata['weights']=data.model.weights
  newdata['tagdict']=data.tagdict
  newdata['classes']=list(data.classes)
  with open(name+".json","w") as json_file:
    json.dump(newdata, json_file, ensure_ascii=False)

In [15]:
convert_model("tud-pt_tagger",pt_tagger)

In [16]:
import json
unigram_tagger_json= json.dumps(unigram_tagger._context_to_tag, ensure_ascii=False)
# Store data (serialize)
with open('tud-unigram_tagger.json', 'w',encoding = "utf-8-sig") as f:
    f.write(unigram_tagger_json)